In [1]:
import os
import sys
import re
sys.path.append("../src")

from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from yt_encoder import YTEncoder
from torch.utils.data import Dataset, DataLoader, RandomSampler
from tqdm import tqdm

import torch
import numpy as np

from pathlib import Path

PATH_TO_DATA = Path("../data")
PATH_TO_MODELS = Path("../models")

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [20]:
tokenizer = YTEncoder.from_pretrained(str(PATH_TO_MODELS / "yt.model"))
model = GPT2LMHeadModel.from_pretrained(str(PATH_TO_MODELS / "s_gpt_2/")).to(device)

In [5]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [6]:
def generate_some_text(input_str, text_len = 250):

    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)

    model.eval()
    with torch.no_grad():

        for i in range(text_len):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=10) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode([output_list])
        print(output_text)

In [26]:
model.load_state_dict(torch.load(str(PATH_TO_MODELS / "gpt2_epoch_4.pt")))

<All keys matched successfully>

In [43]:
generate_some_text(" Не отправилось блядство ")

Не отправилось блядство и наркомания. Их жизнь превратилась в сплошную череда кровавых убийств. В поисках справедливости они вступают в борьбу с преступным миром и попадают в психиатрическую лечебницу. Там они сталкиваются со смертью от рук маньяка из числа бывших наркоманов, который убил всех членов их семьи и забрал у них их собственность, в том числе и их дочь. После того, как маньяк обезглавил их отца, мать и дочь вынуждены скрываться вместе со своим сыном. В это время в больнице работает медсестра, которая должна следить за состоянием матери и дочери. Но вскоре они обнаруживают в стенах психиатрической лечебницы жестокую преступную организацию и их жизнь подвергается опасности со стороны маньяка. Все они подвергаются преследованию и получают серьезные психологические травмы. И тогда они решают объединиться в борьбе с бандой преступников, которая захватила власть в городе и пытается захватить власть в своей стране. Для этих несчастных жителей главное не то, что они могут спасти мир